# 전문가를 위한 파이썬

## part 4

## 챕터8

### 변수는 상자가 아니다

* 객체는 변수가 할당되기 전에 생성된다.
* 기본적으로 파이썬은 변수를 넘길때 call by object reference, call by sharing으로 보내게 된다.
* 매개변수로 받은 변수는 같은 주소 값을 공유하여 mutable일 경우에 값이 변하게 된다.
* 함수 내에서 매개변수와 같은 네이밍의 변수를 선언하면 다른 주소 값을 가리키게 되어 변하지 않게 된다.

In [1]:
a = [1, 2, 3]
b = a
a.append(4)
print(b)

[1, 2, 3, 4]


### 정체성, 동질성, 별명

In [3]:
charles = {'name': 'Charles L. Dodgson', 'born':1832}
lewis = charles
print(lewis is charles)
print(id(charles), id(lewis))
lewis['balance'] = 950
print(charles)

True
2382157658008 2382157658008
{'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}


In [5]:
alex = {'name': 'Charles L. Dodgson', 'born':1832}
print(alex == charles)
print(alex is charles)
print(alex is not charles)
print(charles is lewis)

False
False
True
True


* ```==```연산자가 객체의 값을 비교하는 반면, ```is``` 연산자는 객체의 정체성을 비교한다.
* 변수를 싱글턴과 비교할 때는 ```is``` 연산자를 사용할때는 ```None```과 비교하는 경우가 일반적이다.
* ```is``` 연산자의 경우 ```==```연산자보다 빠르다 하지만 ```a == b```는 ```a.__eq__(b)```의 편리한 구문이다. ```object``` 객체에서 상속받은 ```__eq__()``` 메서드는 ```ID```를 비교하므로 ```is``` 연산자와 같은 결과를 산출한다.

### 기본 복사는 얕은 복사

* `list()`, `dict()`, `set()` 과 같이, `생성자`를 이용하거나, `[:]`와 같이 인덱싱, 슬라이스를 사용하면 얕은 사본을 생성한다.

In [7]:
l1 = [3, [66, 55, 44], (7, 8, 9)]
l2 = list(l1)
l1.append(100)
l1[1].remove(55)
print('l1:', l1)
print('l2:', l2)

l2[1] += [33, 22]
l2[2] += (10, 11)
print('l1:', l1)
print('l2:', l2)

l1: [3, [66, 44], (7, 8, 9), 100]
l2: [3, [66, 44], (7, 8, 9)]
l1: [3, [66, 44, 33, 22], (7, 8, 9), 100]
l2: [3, [66, 44, 33, 22], (7, 8, 9, 10, 11)]


* `deepcopy()`의 경우 깊은 복사를, `copy()`의 경우 얕은 복사를 지원한다.

In [8]:
class Bus:
    
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
            
    def pick(self, name):
        self.passengers.append(name)
    
    def drop(self, name):
        self.passengers.remove(name)

In [9]:
import copy
bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)
print(id(bus1), id(bus2), id(bus3))
bus1.drop('Bill')
print(bus2.passengers)
print(bus3.passengers)

2382157720712 2382157720824 2382157720992
['Alice', 'Claire', 'David']
['Alice', 'Bill', 'Claire', 'David']
2382157693704 2382157693704 2382156756872


* 아래와 같이 bus1과 bus2가 동일한 리스트를 공유하는 것을 알 수 있다.

In [11]:
print(id(bus1.passengers), id(bus2.passengers), id(bus3.passengers))

2382157693704 2382157693704 2382156756872


### del과 가비지 컬렉션

* `del` 명령은 이름을 제거하는 것이지, 객체를 제거하는 것이 아니다. `del` 명령의 결과로 객체가 가비지 컬렉트될 수 있지만, 제거된 변수가 객체를 참조하는 최후의 변수거나 객체에 도달할 수 없을 때만 가비지 컬렉트된다.
* refcount == 0 이 되는 경우 `__del__`을 호출하여 객체까지 제거한다.

In [13]:
import weakref
s1 = {1, 2, 3}
s2 = s1
def bye():
    print('Gone with the wind...')

ender = weakref.finalize(s1, bye)
print(ender.alive)
del s1
print(ender.alive)
s2 = 'spam'
print(ender.alive)

True
True
Gone with the wind...
False


### 약한 참조

* 불필요하게 객체를 유지시키지 않으면서 캨체를 참조할 수 있으면 도움이 되는 경우가 종종 있다. 캐시가 대표적인 경우다.
* 약한 참조는 참조 카운트를 증가시키지 않고 객체를 참조한다.
* 참조의 대상인 객체를 "참조 대상"이라고 한다. 따라서 약한 참조는 참조 대상이 가비지 컬렉트되는 것을 방지하지 않는다고 말할 수 있다.

### 파이썬의 특이한 불변형 처리법

* 예제만...

In [14]:
t1 = (1, 2, 3)
t2 = tuple(t1)
print(t2 is t1)
t3 = t1[:]
print(t3 is t1)

True
True


In [16]:
t1 = (1, 2, 3)
t3 = (1, 2, 3)
print(t3 is t1)

False


In [17]:
s1 = 'ABC'
s2 = 'ABC'
print(s2 is s1)

True


* 요약 : Iid말고 `==`로 비교하자...

### 요약

* 모든 파이썬 객체는 정체성, 자료형, 값을 가지고 있다.
* 코드가 실행되는 동안 객체는 값만 바뀔 뿐이다.
* 변수 두 개가 동일한 값을 가진 불변 객체를 가리키고 있다면, 변수가 각각의 사본을 가리키고 있는지 아니면 동일 객체에 대한 별명인지는 중요하지 않다. -> 어쨌든 불변 객체는 변하지 않기 때문이다. -> 하지만 튜플이나 frozenset 등의 불변 컬렉션인 경우는 예외이다.